In [1]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from xgboost import XGBClassifier

import pandas as pd
import pickle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy    

# read correct ansers
Location = r'/home/stratos/Desktop/ted_Project/twitter_data/SemEval2017_task4_subtaskA_test_english_gold.txt'
df = pd.read_csv(Location, error_bad_lines=False,
                    names=['no1','cent'],sep='\t')
# df=df[:-1]
# ytrain=df.drop(columns=['no1','no2','tweet'])

# read test_data
infile = open("lemmatized_tweets_test.pickle",'rb')
test_data = pickle.load(infile)
infile.close()

# # concatinate tokens. ending up with a filtered tweet
tok=[]
for tweet in test_data:
    tmp=''
    for word in tweet:
        tmp=tmp+' '+word
    tok.append(tmp)
test_data=tok


sz=12000
attributes=1500


test_vectori=[
    [CountVectorizer(max_features=attributes)],
    [TfidfVectorizer(max_features=attributes)]
]

test=[
    [neighbors.KNeighborsClassifier(n_jobs=4,algorithm='brute',weights='distance')],
    [LogisticRegression(n_jobs=4)],
    [RandomForestClassifier(n_jobs=4,n_estimators=250)],
    [svm.SVC(kernel='linear', C=1, probability=True)],
    [XGBClassifier()]
]


test1=[x+test_vectori[0] for x in test]
test2=[x+test_vectori[1] for x in test]

test=test1+test2   


prediction=[]
for classifier, vectori in test:
    _vectorizer = vectori
    _xtrain = _vectorizer.fit_transform(test_data)
    
    file_name=classifier.__class__.__qualname__ +"_"+vectori.__class__.__qualname__+"_"+str(attributes)+".pickle"
    # read w2v
    infile = open(file_name,'rb')
    classifi = pickle.load(infile)
    infile.close()

    print("loaded "+file_name)
    
    tmp=classifi.predict(_xtrain[:sz])
    prediction.append(tmp) #predict on the validation set
    print("classifier= "+classifier.__class__.__qualname__ +"\tvectorizer= "+vectori.__class__.__qualname__+"\tscore= ",end=" ")
    print(f1_score(df['cent'][:sz], tmp,average='macro'))#evaluate on the validation set
    

def most_frequent(List): 
    return max(set(List), key = List.count) 
    
#voting prosses
ans=[]
for i in range (sz):
    list2 = [item[i] for item in prediction]
    ans.append(most_frequent(list2))
ans=numpy.asarray(ans)

print("all classifiers score= ",end=" ")
print(f1_score(df['cent'][:sz], ans,average='macro'))#evaluate on the validation set


/home/stratos/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


loaded KNeighborsClassifier_CountVectorizer_1500.pickle
classifier= KNeighborsClassifier	vectorizer= CountVectorizer	score=  0.32539820928914326
loaded LogisticRegression_CountVectorizer_1500.pickle
classifier= LogisticRegression	vectorizer= CountVectorizer	score=  0.32107406523026466
loaded RandomForestClassifier_CountVectorizer_1500.pickle
classifier= RandomForestClassifier	vectorizer= CountVectorizer	score=  0.3247492972804286
loaded SVC_CountVectorizer_1500.pickle
classifier= SVC	vectorizer= CountVectorizer	score=  0.32330871341998485
loaded XGBClassifier_CountVectorizer_1500.pickle


/home/stratos/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


classifier= XGBClassifier	vectorizer= CountVectorizer	score=  0.30414008879148874
loaded KNeighborsClassifier_TfidfVectorizer_1500.pickle
classifier= KNeighborsClassifier	vectorizer= TfidfVectorizer	score=  0.32552781265075464
loaded LogisticRegression_TfidfVectorizer_1500.pickle
classifier= LogisticRegression	vectorizer= TfidfVectorizer	score=  0.32005712869819064
loaded RandomForestClassifier_TfidfVectorizer_1500.pickle
classifier= RandomForestClassifier	vectorizer= TfidfVectorizer	score=  0.3246682752457179
loaded SVC_TfidfVectorizer_1500.pickle
classifier= SVC	vectorizer= TfidfVectorizer	score=  0.3212417411032377
loaded XGBClassifier_TfidfVectorizer_1500.pickle
classifier= XGBClassifier	vectorizer= TfidfVectorizer	score=  0.3025013254009077
all classifiers score=  0.3192574253091575


/home/stratos/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
